In [1]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec, KeyedVectors
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model,save_model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import layers
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense,LSTM,Embedding,Input,Lambda,Concatenate,Dropout,Activation,Flatten,Bidirectional,Layer,Attention
from keras.optimizers.legacy import Adam, SGD,RMSprop
import math
from keras.layers import Activation
from keras.utils import get_custom_objects, custom_object_scope
import keras.backend as K

import re
nltk.download('punkt')
from sklearn.model_selection import train_test_split
import itertools
import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


> # Data yang digunakan

In [2]:
#@ini data yang dipakai
word2vecModel = "/kaggle/input/question-answering/dataset/word2vec2.model"
dataset_final =  "/kaggle/input/question-answering/dataset/dataset_final.json"
after_prepro = "/kaggle/input/question-answering/dataset/after prepro.json"

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/question-answering/dataset/word2vec2.model
/kaggle/input/question-answering/dataset/dataset_final.json
/kaggle/input/question-answering/dataset/after prepro.json


In [4]:
df = pd.read_json(dataset_final)
df['answer_pad']=''
df['kandidat']	=''
df['kandidat_pad']=''
df['bad_answer']=''
df

,question,answer,answer_pad,kandidat,kandidat_pad,bad_answer
0,Apa penyebab turunnya omzet penjualan bunga hi...,"Enjang, seorang pedagang bunga, menduga penuru...",,,,
1,Apa yang dilakukan pendaki gunung untuk memper...,"Bagi yang mempunyai hobi mendaki gunung, merek...",,,,
2,Mengapa nama kerta api Empu Jaya akan diganti?,"Usut punya usut, nama Empu Jaya dianggap terla...",,,,
3,Apa penyebab terjadinya kelangkaan minyak di b...,Penyebabnya adalah kepanikan warga atas rencan...,,,,
4,Mengapa Muchyar Yara dihukum?,Dia diberi sanksi lantaran mengkritik kepemimp...,,,,
...,...,...,...,...,...,...
595,Mengapa mobil otonom menjadi populer?,Mobil otonom menjadi populer karena dapat meni...,,,,
596,Bagaimana cara memilih oli mesin yang tepat?,Beberapa faktor yang perlu dipertimbangkan saa...,,,,
597,Apa saja manfaat dari teknologi telematika pad...,Teknologi telematika pada kendaraan memiliki b...,,,,
598,Bagaimana cara merawat kulit wajah agar tetap ...,Beberapa cara yang dapat dilakukan untuk meraw...,,,,


In [5]:
for index, row in df.iterrows():
    df.at[index,'question'] = gensim.utils.simple_preprocess(row['question'])
    df.at[index,'answer'] = gensim.utils.simple_preprocess(row['answer'])
    df.at[index,'answer_pad'] = gensim.utils.simple_preprocess(row['answer'])
df.head()

,question,answer,answer_pad,kandidat,kandidat_pad,bad_answer
0,"[apa, penyebab, turunnya, omzet, penjualan, bu...","[enjang, seorang, pedagang, bunga, menduga, pe...","[enjang, seorang, pedagang, bunga, menduga, pe...",,,
1,"[apa, yang, dilakukan, pendaki, gunung, untuk,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,...",,,
2,"[mengapa, nama, kerta, api, empu, jaya, akan, ...","[usut, punya, usut, nama, empu, jaya, dianggap...","[usut, punya, usut, nama, empu, jaya, dianggap...",,,
3,"[apa, penyebab, terjadinya, kelangkaan, minyak...","[penyebabnya, adalah, kepanikan, warga, atas, ...","[penyebabnya, adalah, kepanikan, warga, atas, ...",,,
4,"[mengapa, muchyar, yara, dihukum]","[dia, diberi, sanksi, lantaran, mengkritik, ke...","[dia, diberi, sanksi, lantaran, mengkritik, ke...",,,


In [6]:
data_list = []
for i in range(0, 600, 20):
    group = list(range(i, i+20))
    data_list.append(group)
new_data=[['']]*600
for i in range(0,600):
    x = math.ceil((i+1)/20)
    y = x-1
    new_data[i]=data_list[y]
heil = pd.DataFrame(columns=['kelompok'],index=range(len(new_data)))
for i in range(len(new_data)):
  heil.at[i,'kelompok']= new_data[i]

In [7]:
def tukar(lists,cari):
  salinan = lists[:]
  if cari in salinan:
    index = salinan.index(cari)
    if index == 0:
      return salinan
    else:
      salinan[0],salinan[index] = salinan[index],salinan[0]
      return salinan

In [8]:
import random
for index,row in heil.iterrows():
  heil.at[index,'kelompok'] = tukar(row['kelompok'],index)
heil['bad']=''
heil['good']=''
for index,row in heil.iterrows():
  heil.at[index,'kelompok'] = [row['kelompok'][0]]+ random.sample(row['kelompok'][1:],len(row['kelompok'])-1)
  heil.at[index,'bad']=int(row['kelompok'][1])
  heil.at[index,'good']=int(row['kelompok'][0])
heil.head()

,kelompok,bad,good
0,"[0, 11, 9, 15, 8, 7, 10, 6, 19, 17, 5, 1, 13, ...",1,0
1,"[1, 11, 0, 9, 18, 14, 16, 12, 15, 8, 13, 7, 17...",0,1
2,"[2, 0, 12, 17, 14, 3, 19, 6, 4, 9, 10, 8, 15, ...",1,2
3,"[3, 16, 19, 1, 6, 14, 2, 11, 7, 4, 18, 15, 13,...",1,3
4,"[4, 2, 17, 3, 19, 0, 18, 13, 7, 10, 5, 9, 1, 1...",1,4


In [9]:
def kandidat(baris):
    text =[]
    for i in baris:
        text.append(df['answer'][i])
    return text

In [10]:
heil['kandidat'] = ''
for index,row in heil.iterrows():
    heil.at[index,'kandidat']=kandidat(row['kelompok'])

In [11]:
heil

,kelompok,bad,good,kandidat
0,"[0, 11, 9, 15, 8, 7, 10, 6, 19, 17, 5, 1, 13, ...",1,0,"[[enjang, seorang, pedagang, bunga, menduga, p..."
1,"[1, 11, 0, 9, 18, 14, 16, 12, 15, 8, 13, 7, 17...",0,1,"[[bagi, yang, mempunyai, hobi, mendaki, gunung..."
2,"[2, 0, 12, 17, 14, 3, 19, 6, 4, 9, 10, 8, 15, ...",1,2,"[[usut, punya, usut, nama, empu, jaya, diangga..."
3,"[3, 16, 19, 1, 6, 14, 2, 11, 7, 4, 18, 15, 13,...",1,3,"[[penyebabnya, adalah, kepanikan, warga, atas,..."
4,"[4, 2, 17, 3, 19, 0, 18, 13, 7, 10, 5, 9, 1, 1...",1,4,"[[dia, diberi, sanksi, lantaran, mengkritik, k..."
...,...,...,...,...
595,"[595, 580, 596, 584, 592, 594, 583, 581, 590, ...",581,595,"[[mobil, otonom, menjadi, populer, karena, dap..."
596,"[596, 598, 597, 588, 586, 584, 580, 592, 591, ...",581,596,"[[beberapa, faktor, yang, perlu, dipertimbangk..."
597,"[597, 599, 590, 585, 592, 598, 582, 586, 596, ...",581,597,"[[teknologi, telematika, pada, kendaraan, memi..."
598,"[598, 599, 586, 585, 580, 595, 589, 596, 592, ...",581,598,"[[beberapa, cara, yang, dapat, dilakukan, untu..."


In [12]:
df['kandidat']=heil['kandidat']
df['kandidat_pad']=heil['kandidat']

In [13]:
for index,row in df.iterrows():
  df.at[index,'bad_answer'] = df['answer'][heil['bad'][index]]

In [14]:
df.head()

,question,answer,answer_pad,kandidat,kandidat_pad,bad_answer
0,"[apa, penyebab, turunnya, omzet, penjualan, bu...","[enjang, seorang, pedagang, bunga, menduga, pe...","[enjang, seorang, pedagang, bunga, menduga, pe...","[[enjang, seorang, pedagang, bunga, menduga, p...","[[enjang, seorang, pedagang, bunga, menduga, p...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."
1,"[apa, yang, dilakukan, pendaki, gunung, untuk,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,...","[[bagi, yang, mempunyai, hobi, mendaki, gunung...","[[bagi, yang, mempunyai, hobi, mendaki, gunung...","[enjang, seorang, pedagang, bunga, menduga, pe..."
2,"[mengapa, nama, kerta, api, empu, jaya, akan, ...","[usut, punya, usut, nama, empu, jaya, dianggap...","[usut, punya, usut, nama, empu, jaya, dianggap...","[[usut, punya, usut, nama, empu, jaya, diangga...","[[usut, punya, usut, nama, empu, jaya, diangga...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."
3,"[apa, penyebab, terjadinya, kelangkaan, minyak...","[penyebabnya, adalah, kepanikan, warga, atas, ...","[penyebabnya, adalah, kepanikan, warga, atas, ...","[[penyebabnya, adalah, kepanikan, warga, atas,...","[[penyebabnya, adalah, kepanikan, warga, atas,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."
4,"[mengapa, muchyar, yara, dihukum]","[dia, diberi, sanksi, lantaran, mengkritik, ke...","[dia, diberi, sanksi, lantaran, mengkritik, ke...","[[dia, diberi, sanksi, lantaran, mengkritik, k...","[[dia, diberi, sanksi, lantaran, mengkritik, k...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."


In [15]:
def list2text(data):
  listing = []
  for i in range(len(data)):
    kalimat = ' '.join([str(item) for item in data[i]])
    listing.append(kalimat)
    # listing.append(kalimat)
  return listing

In [16]:
def cekmax(data):
    dat = data.str.len().max()
    return dat

In [17]:
pertanyaan = cekmax(df["question"])
jawaban = cekmax(df["answer"])
print(" panjang pertanyaan : "+str(pertanyaan)+","+" panjang jawaban : "+str(jawaban))

 panjang pertanyaan : 14, panjang jawaban : 57


In [18]:
def padding(data,atr,length):
  if atr == 'kandidat_pad':
    for i, kandidat in enumerate(data[atr]):
      al = []
      for j in kandidat:
        # print(len(j))
        j+=['<PAD>']*(length-len(j))
        al.append(j)
      data.at[i,atr] = al
  else:
    for i in range(len(data)):
      data.at[i,atr]+=['<PAD>']*(length-len(data.at[i,atr]))

In [19]:
padding(df,'question',15)
padding(df,'answer_pad',60)
padding(df,'bad_answer',60)
padding(df,'kandidat_pad',60)
df.head()

,question,answer,answer_pad,kandidat,kandidat_pad,bad_answer
0,"[apa, penyebab, turunnya, omzet, penjualan, bu...","[enjang, seorang, pedagang, bunga, menduga, pe...","[enjang, seorang, pedagang, bunga, menduga, pe...","[[enjang, seorang, pedagang, bunga, menduga, p...","[[enjang, seorang, pedagang, bunga, menduga, p...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."
1,"[apa, yang, dilakukan, pendaki, gunung, untuk,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,...","[[bagi, yang, mempunyai, hobi, mendaki, gunung...","[[bagi, yang, mempunyai, hobi, mendaki, gunung...","[enjang, seorang, pedagang, bunga, menduga, pe..."
2,"[mengapa, nama, kerta, api, empu, jaya, akan, ...","[usut, punya, usut, nama, empu, jaya, dianggap...","[usut, punya, usut, nama, empu, jaya, dianggap...","[[usut, punya, usut, nama, empu, jaya, diangga...","[[usut, punya, usut, nama, empu, jaya, diangga...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."
3,"[apa, penyebab, terjadinya, kelangkaan, minyak...","[penyebabnya, adalah, kepanikan, warga, atas, ...","[penyebabnya, adalah, kepanikan, warga, atas, ...","[[penyebabnya, adalah, kepanikan, warga, atas,...","[[penyebabnya, adalah, kepanikan, warga, atas,...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."
4,"[mengapa, muchyar, yara, dihukum, <PAD>, <PAD>...","[dia, diberi, sanksi, lantaran, mengkritik, ke...","[dia, diberi, sanksi, lantaran, mengkritik, ke...","[[dia, diberi, sanksi, lantaran, mengkritik, k...","[[dia, diberi, sanksi, lantaran, mengkritik, k...","[bagi, yang, mempunyai, hobi, mendaki, gunung,..."


In [20]:
df.to_json("//kaggle/working//after prepro 2.json")
print('sukses')

sukses


In [21]:
kalimat = []
for i in range(len(df)):
  kalimat.append(df['question'][i])
  kalimat.append(df['answer_pad'][i])

In [22]:
model_w2v = Word2Vec(sentences=kalimat,vector_size=100, window=8, min_count=1,sg=1, workers=8,alpha=0.025,min_alpha=0.0001)
model_w2v.train(kalimat, total_examples=len(kalimat), compute_loss=True, epochs=50,)
model_w2v.wv.save_word2vec_format("/kaggle/working/word2vec 2nd.model",binary=False)

# Pakai ini ketika sudah pernah melakukan preprocessing sebelumnya

In [23]:
df = pd.read_json(after_prepro)
model_w2v = KeyedVectors.load_word2vec_format(word2vecModel,binary=False)

In [24]:
kandidat = []
kandidat_pad = []
question = []
answer = []
bad_answer = []
for i, j in df.iterrows():
  kandidat.append(j['kandidat'])
  kandidat_pad.append(j['kandidat_pad'])
  question.append(j['question'])
  answer.append(j['answer_pad'])
  bad_answer.append(j['bad_answer'])

In [25]:
question_filter = []
answer_filter=[]
bad_answer_filter = []
kandidat_pad_filter =[]
for c in question:
  word_filter=[]
  for word in c:
    # if word in model_w2v.vocab:
    if word in model_w2v.key_to_index:
      word_filter.append(word)
  if len(word_filter) != 0:
    question_filter.append(word_filter)

for c in answer:
  word_filter=[]
  for word in c:
    if word in model_w2v.key_to_index:
      word_filter.append(word)
  if len(word_filter) != 0:
    answer_filter.append(word_filter)
for c in bad_answer:
  word_filter=[]
  for word in c:
    if word in model_w2v.key_to_index:
      word_filter.append(word)
  if len(word_filter) != 0:
    bad_answer_filter.append(word_filter)
def filtering_words(words):
  filtered=[]
  for c in words:
    word_filter=[]
    for word in c:
      if word in model_w2v.key_to_index:
        word_filter.append(word)
    if len(word_filter)!=0:
      filtered.append(word_filter)
  return filtered
for c in kandidat_pad:
  filtering = []
  for d in c:
    word_filter=[]
    for word in d:
      if word in model_w2v.key_to_index:
        word_filter.append(word)
    if len(word_filter) != 0:
      filtering.append(word_filter)
  if len(filtering)!=0:
    kandidat_pad_filter.append(filtering)

In [26]:
new_bad = []
for x in bad_answer_filter:
  new = x +['<PAD>']*(60-len(x))
  new_bad.append(new)
bad_answer_filter = new_bad

In [27]:
question_vector = []
answer_vector = []
badanswer_vector = []
kandidat_pad_vector = []
for c in question_filter:
  question_vector.extend([model_w2v[word] for word in c])
for c in answer_filter:
  answer_vector.extend([model_w2v[word] for word in c])
for c in bad_answer_filter:
  badanswer_vector.extend([model_w2v[word] for word in c])
for c in kandidat_pad_filter:
  filtering = []
  for d in c:
    filtering.extend([model_w2v[word] for word in d])
  kandidat_pad_vector.append(filtering)

# Modifikasi data

In [28]:
question_index =[]
answer_index = []
badanswer_index = []
kandidat_pad_index = []
for c in question_filter:
  question_index.append([model_w2v.key_to_index[word] for word in c])
for c in answer_filter:
  answer_index.append([model_w2v.key_to_index[word] for word in c])
for c in bad_answer_filter:
  badanswer_index.append([model_w2v.key_to_index[word] for word in c])
for c in kandidat_pad_filter:
  filtering = []
  for d in c:
    filtering.append([model_w2v.key_to_index[word] for word in d])
  kandidat_pad_index.append(filtering)

In [29]:
df2 = pd.DataFrame({'question':['']*len(df),'answer':['']*len(df),'bad_answer':['']*len(df),'kandidat':['']*len(df)})
for i in range(len(df2)):
  df2.at[i,'question'] = question_index[i]
  df2.at[i,'answer'] = answer_index[i]
  df2.at[i,'bad_answer'] = badanswer_index[i]
  df2.at[i,'kandidat'] =kandidat_pad_index[i]

In [30]:
embedding_matrik = np.zeros((len(model_w2v.key_to_index),100))
for i in range(len(model_w2v.key_to_index)):
  embedding_vector = model_w2v[model_w2v.index_to_key[i]]
  if embedding_vector is not None:
    embedding_matrik[i] = embedding_vector

In [31]:
train_data, test_data = train_test_split(df2,test_size=0.3,random_state=42)

In [32]:
train_question,train_answer,train_badanswer,test_question,test_answer,test_badanswer,test_kandidat,train_kandidat = [[] for i in range(8)]

for i in train_data['question']:
  train_question.append(i)
for i in train_data['answer']:
  train_answer.append(i)
for i in train_data['bad_answer']:
  train_badanswer.append(i)
for i in train_data['kandidat']:
  train_kandidat.append(i)
for i in test_data['question']:
  test_question.append(i)
for i in test_data['answer']:
  test_answer.append(i)
for i in test_data['bad_answer']:
  test_badanswer.append(i)
for i in test_data['kandidat']:
  test_kandidat.append(i)

In [33]:
train_question = np.asarray(train_question,dtype=int)
train_answer = np.asarray(train_answer,dtype=int)
train_badanswer = np.asarray(train_badanswer,dtype=int)
train_kandidat = np.asarray(train_kandidat,dtype=int)
test_question = np.asarray(test_question,dtype=int)
test_answer = np.asarray(test_answer,dtype=int)
test_badanswer = np.asarray(test_badanswer,dtype=int)
test_kandidat = np.asarray(test_kandidat,dtype=int)

In [34]:
class QaModel():
  def __init__(self,units,dropout,optimizers,nama,attentions=False):
    self.units=units
    self.dropout=dropout
    self.optimizers=optimizers
    self.nama=nama
    self.attentions=attentions

  def cosine_similarity(self):
    return lambda x: K.batch_dot(x[0], x[1], axes=1) / K.maximum(K.sqrt(K.batch_dot(x[0], x[0]) * K.batch_dot(x[1], x[1])), K.epsilon())

  def max_pooling(self,lstm_out):
    output = tf.reduce_max(lstm_out, axis=1)
    return output
  @keras.saving.register_keras_serializable(package="my_package", name="custom_loss")
  def custom_loss(self,y_true,y_pred):
      return y_pred

  @keras.saving.register_keras_serializable(package="my_package", name="custom_attention")
  def attention(self,input_q,input_a):
    h = self.units*2
    w = int(input_q.get_shape()[2]) #panjang dari input untuk tep pertama
    h_a = int(input_a.get_shape()[1]) #panjang dari input untuk jawaban
    print('w',w)
    # print('h_q',h_q)
    print('h_a',h_a)
    output_q = self.max_pooling(input_q) #maxpooling layer dengan output #(b,w)
    print('output_q.shape 0',output_q.shape)

    reshape_q = tf.expand_dims(output_q,1) #(b,1,w) mengubah bentuk dari output_q
    print('reshape_q.shape 1',reshape_q.shape)
    reshape_q = tf.tile(reshape_q,[1,h_a,1]) #(b,h_a,1)
    print('reshape_q.shape 2',reshape_q.shape)
    reshape_q = tf.reshape(reshape_q,[-1,w]) #(n*h_a,w)
    print('reshape_q.shape 3',reshape_q.shape)
    reshape_a = tf.reshape(input_a,[-1,w]) #(b*h_a,w)
    print('reshape_q.shape 4',reshape_q.shape)

    # with tf.name_scope('my_variable',) as scope:
    Wam = tf.random.truncated_normal([h,h], stddev=0.1)
    Wqm = tf.random.truncated_normal([h,h], stddev=0.1)
    Wms = tf.random.truncated_normal([h,1], stddev=0.1)

    Wqm = tf.matmul(reshape_q,Wqm)
    print('Wqm.shape',Wqm.shape)
    Wam = tf.matmul(reshape_a,Wam)
    print('Wam.shape',Wam.shape)
    Wsum = tf.add(Wqm,Wam)
    print('Wsum.shape',Wsum.shape)
    M = tf.tanh(Wsum)
    print('M 1',M.shape)
    M = tf.matmul(M,Wms) #(b*h_a,1)
    print('M 2',M.shape)

    S = tf.reshape(M,[-1,h_a]) #(b,h_a)
    print('S 1',S.shape)
    S = tf.nn.softmax(S) #(b,h_a)
    print('S 2',S.shape)

    S_diag = tf.linalg.diag(S) #(b,h_a,h_a)
    print('S_diag',S_diag.shape)
    attention_a = tf.matmul(S_diag, input_a) #(b,h_a,w)
    print('attention_a',attention_a.shape)

    output_a = self.max_pooling(attention_a) #(b,w)
    print('output a ',output_a)
    return tf.tanh(output_q),tf.tanh(output_a)

  def get_bilstm_model(self):
    max_question = 15
    max_answer = 60
    margin=0.1

    question = Input(shape=(max_question,),dtype='int32',name='question_base')
    answer = Input(shape=(max_answer,), dtype='int32', name='answer')
    answer_good = Input(shape=(max_answer,), dtype='int32', name='answer_good_base')
    answer_bad = Input(shape=(max_answer,), dtype='int32', name='answer_bad_base')

    qa_embedding = Embedding(input_dim=embedding_matrik.shape[0],output_dim=embedding_matrik.shape[1],mask_zero=False,weights=[embedding_matrik],trainable=False)
    bilstm = Bidirectional(LSTM(units=self.units, dropout=self.dropout, return_sequences=True))

    answer_embedding =qa_embedding(answer)
    print('answer embedding 1',answer_embedding.shape)
    answer_embedding = bilstm(answer_embedding)
    print('bi lstm 1',answer_embedding.shape)

    question_embedding =qa_embedding(question)
    print('question embedding',question_embedding.shape)
    question_embedding = bilstm(question_embedding)
    print('bi lstm 2',question_embedding.shape)

    if (self.attentions):
      q_att,a_att = self.attention(question_embedding,answer_embedding)
      print('aatt',a_att.shape)
      print('qatt',q_att.shape)
      similarity = self.cosine_similarity()
      question_answer_similarity = Lambda(similarity)([q_att, a_att])
    else:
      answer_encoded = self.max_pooling(answer_embedding)
      question_encoded = self.max_pooling(question_embedding)
      print('ans ',answer_encoded.shape)
      print('que',question_encoded.shape)
      similarity = self.cosine_similarity()
      question_answer_similarity = Lambda(similarity)([question_encoded, answer_encoded])

    bilstm_model = Model(name='bilstm',inputs=[question,answer],outputs=question_answer_similarity)
    good_similarity = bilstm_model([question,answer_good])
    bad_similarity = bilstm_model([question,answer_bad])

    loss = Lambda(lambda x: K.relu(margin -x[0]+x[1]),lambda x: x[0])([good_similarity,bad_similarity])


    # custom_loss =Lambda(lambda y_true,y_pred:y_pred)

    training_model = Model(inputs=[question, answer_good, answer_bad], outputs=loss, name='training_model')
    training_model.compile(loss=self.custom_loss, optimizer=self.optimizers, metrics=["acc"])
    prediction_model = Model(inputs=[question, answer_good], outputs=good_similarity, name='prediction_model')
    prediction_model.compile(loss=self.custom_loss, optimizer=self.optimizers, metrics=["acc"])
    return training_model, prediction_model,self.nama


In [35]:
Y = np.full((train_question.shape[0],),0)
y = np.full((test_question.shape[0],),0)

In [36]:
model = QaModel(75,0.1,RMSprop(learning_rate=0.01),'uji terbaik',True)
train,predik,nama = model.get_bilstm_model()

answer embedding 1 (None, 60, 100)
bi lstm 1 (None, 60, 150)
question embedding (None, 15, 100)
bi lstm 2 (None, 15, 150)
w 150
h_a 60
output_q.shape 0 (None, 150)
reshape_q.shape 1 (None, 1, 150)
reshape_q.shape 2 (None, 60, 150)
reshape_q.shape 3 (None, 150)
reshape_q.shape 4 (None, 150)
Wqm.shape (None, 150)
Wam.shape (None, 150)
Wsum.shape (None, 150)
M 1 (None, 150)
M 2 (None, 1)
S 1 (None, 60)
S 2 (None, 60)
S_diag (None, 60, 60)
attention_a (None, 60, 150)
output a  KerasTensor(type_spec=TensorSpec(shape=(None, 150), dtype=tf.float32, name=None), name='tf.math.reduce_max_1/Max:0', description="created by layer 'tf.math.reduce_max_1'")
aatt (None, 150)
qatt (None, 150)


In [37]:
filename = "/kaggle/working/model terbaik.hdf5"
checkpoint = ModelCheckpoint(filepath=filename,monitor='val_loss',verbose=1, save_best_only=True,save_weights_only=True,mode='min')
train.fit([train_question,train_answer,train_badanswer],Y,validation_data=([test_question,test_answer,test_badanswer],y),callbacks=checkpoint,epochs=100)
train.save_weights("/kaggle/working/train terbaik.h5", overwrite=True)
predik.save_weights("/kaggle/working/predik terbaik.h5", overwrite=True)

Epoch 1/100
12/14 [========================>.....] - ETA: 0s - loss: 0.0411 - acc: 1.0000
Epoch 1: val_loss improved from inf to 0.02739, saving model to /kaggle/working/model terbaik.hdf5
14/14 [==============================] - 18s 266ms/step - loss: 0.0389 - acc: 1.0000 - val_loss: 0.0274 - val_acc: 1.0000
Epoch 2/100
13/14 [==========================>...] - ETA: 0s - loss: 0.0147 - acc: 1.0000
Epoch 2: val_loss did not improve from 0.02739
14/14 [==============================] - 0s 29ms/step - loss: 0.0146 - acc: 1.0000 - val_loss: 0.0287 - val_acc: 1.0000
Epoch 3/100
13/14 [==========================>...] - ETA: 0s - loss: 0.0072 - acc: 1.0000
Epoch 3: val_loss did not improve from 0.02739
14/14 [==============================] - 0s 25ms/step - loss: 0.0072 - acc: 1.0000 - val_loss: 0.0351 - val_acc: 1.0000
Epoch 4/100
13/14 [==========================>...] - ETA: 0s - loss: 0.0056 - acc: 1.0000
Epoch 4: val_loss improved from 0.02739 to 0.02429, saving model to /kaggle/working/m

In [38]:
def filtering_words(words):
  filtered=[]
  for c in words:
    word_filter=[]
    for word in c:
      if word in model_w2v.key_to_index:
        word_filter.append(word)
    if len(word_filter)!=0:
      filtered.append(word_filter)
  return filtered
def indexing_words(words):
  indexed = []
  for c in words:
    indexed.append([model_w2v.key_to_index[word] for word in c])
  return indexed

In [39]:
class QAData():
  def __init__(self):
    self.model = KeyedVectors.load_word2vec_format(word2vecModel)
    self.max_question = 15
    self.max_answer = 60
  def pad(self, data, length):
    return pad_sequences(data, maxlen=length, padding='post', truncating='post', value=0)

  def process_test_data(self, question, answers):
    answers = self.pad(answers, self.max_answer)
    question = self.pad(question, self.max_question)
    return question, answers

In [40]:
len(test_question)

180

In [41]:
from scipy.stats import rankdata
c=0
c1=0
for i in range(len(test_question)):
    question = [test_question[i]]*20
    answer = test_kandidat[i]
    qa_data = QAData()
    question_data,answer_data=qa_data.process_test_data(question, answer)
    with tf.device('/GPU:0'):
        sims = predik.predict([question_data,answer_data])
        max_r = np.argmax(sims)
        max_n = 0
        sim_t = sims[0]
        r = rankdata(-sims)
        c+=1 if max_r == max_n else 0
        c1+=1/float(r[max_n]-r[max_r]+1)
MRR = c1/float(len(test_question))
MAP = c/float(len(test_question))

1/1 [==============================] - 0s 24ms/step


In [42]:
print('nilai MRR = ',MRR)
print('nilai MAP = ',MAP)

nilai MRR =  0.7735570008241579
nilai MAP =  0.6777777777777778


In [43]:
# Menggabungkan array 2D
merged_kandidat = np.concatenate((test_kandidat, train_kandidat), axis=0)
print(merged_kandidat.shape)
merged_question = np.concatenate((test_question, train_question), axis=0)
print(merged_question.shape)


(600, 20, 60)
(600, 15)


In [44]:
c=0
c1=0
for i in range(len(merged_question)):
    question = [merged_question[i]]*20
    answer = merged_kandidat[i]
    qa_data = QAData()
    question_data,answer_data=qa_data.process_test_data(question, answer)
    with tf.device('/GPU:0'):
        sims = predik.predict([question_data,answer_data])
        max_r = np.argmax(sims)
        max_n = 0
        sim_t = sims[0]
        r = rankdata(-sims)
        c+=1 if max_r == max_n else 0
        c1+=1/float(r[max_n]-r[max_r]+1)
MRR2 = c1/float(len(merged_question))
MAP2 = c/float(len(merged_question))

1/1 [==============================] - 0s 23ms/step


In [45]:
print('mrr : ', MRR2)
print('map : ', MAP2)

mrr :  0.8401663065964534
map :  0.7516666666666667
